In [1]:
import netCDF4 as nc
import numpy as np
from datetime import datetime as dt

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
%matplotlib inline
rcParams['figure.figsize'] = (15.,10)


In [2]:
#File and dir
IN_DIR='/users/global/albmar/CarboBiocrop/Brattleby/data/'
IN_FILE=IN_DIR+'Marta+Brattleby_arable_flux_2012-12.dat'

OUTFILE='/prj/GREENHOUSE/GREENHOUSE_sites/data/Brattleby/ConCrop_FullFlux_data_albmar.nc'



In [3]:
INlines=open(IN_FILE,'r').readlines()
headers=INlines.pop(0).split('\t')
print(headers)
units=INlines.pop(0).split('\t')
print(units)

['Date', 'Gap filled sensible heat flux', 'Gap filled latent heat flux', 'Fraction measured latent heat present ', 'Gap filled NEE', 'fraction measured NEE present', 'Total ecosystem respiration', 'GPP \n']
['', 'W m-2', 'W m-2', '-', 'mmol CO2 m-2 s-1', '-', 'mmol CO2 m-2 s-1', 'mmol CO2 m-2 s-1\n']


In [4]:
wk_hdrs=['Date','SHF','LHF','LHF_frac','NEE','NEE_frac','TER','GPP']

In [19]:
Data_Dict={ hdr:[] for hdr in wk_hdrs}
for line in INlines:
    split=line[:-1].split('\t')
    for hdr,val in zip(wk_hdrs,split):
        if hdr=='Date':
            Data_Dict[hdr].append(val)
        else:
            Data_Dict[hdr].append(float(val))

Data_Dict['Date'] =np.array( [dt.strptime(date,'%d-%b-%Y') for date in Data_Dict['Date'] ] )
Date=Data_Dict['Date']
del Data_Dict['Date']

for hdr in wk_hdrs[1:]:
    Data_Dict[hdr]=np.array(Data_Dict[hdr])


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [22]:

Date_Num = nc.date2num(Date,units='days since 2012-01-01 00:00:00')
Data_Dict['date']=Date_Num

In [23]:
DataFrame=pd.DataFrame(Data_Dict)#,index=Date_Num)

In [45]:
outf.close()

In [30]:
ENCODING= { wk_hdrs[ihdr]: {'dtype':'float32','fill_value':-9999.,'units':units[ihdr]} \
                for ihdr in range(1,len(wk_hdrs)) }
ENCODING['date']={'dtype':'float32','units':'days since 2012-01-01 00:00:00'}

In [46]:
outf=nc.Dataset(OUTFILE,'w')

outf.createDimension('date',len(Date_Num))
for var in Data_Dict:
    outvar=outf.createVariable(var,'float32',('date'),fill_value=-9999)
    for att in ENCODING[var]:
        outvar.setncattr(att,ENCODING[var][att])
    outvar[:]=Data_Dict[var][:]

outf.title='Brattelby Flux Tower Data'
outf.owner='Edward Comyn-Platt'
outf.close()


In [34]:
DF_Xarr=DataFrame.to_xarray()
DF_Xarr.to_netcdf(OUTFILE)#,encoding=ENCODING)

In [33]:
help(DF_Xarr.to_netcdf)

Help on method to_netcdf in module xarray.core.dataset:

to_netcdf(path=None, mode='w', format=None, group=None, engine=None, encoding=None) method of xarray.core.dataset.Dataset instance
    Write dataset contents to a netCDF file.
    
    Parameters
    ----------
    path : str, optional
        Path to which to save this dataset. If no path is provided, this
        function returns the resulting netCDF file as a bytes object; in
        this case, we need to use scipy.io.netcdf, which does not support
        netCDF version 4 (the default format becomes NETCDF3_64BIT).
    mode : {'w', 'a'}, optional
        Write ('w') or append ('a') mode. If mode='w', any existing file at
        this location will be overwritten.
    format : {'NETCDF4', 'NETCDF4_CLASSIC', 'NETCDF3_64BIT', 'NETCDF3_CLASSIC'}, optional
        File format for the resulting netCDF file:
    
        * NETCDF4: Data is stored in an HDF5 file, using netCDF4 API
          features.
        * NETCDF4_CLASSIC: Data 